In [ ]:
import pandas as pd

In [ ]:
data_df = pd.read_csv('UserBehavior.csv', header=None)
data_df

In [ ]:
data_df = data_df.groupby(by=[0, 1, 2], as_index=False)[3].agg(list)

In [ ]:
data_df.dropna(inplace=True)
data_df['click'] = data_df[3].map(lambda x: int('pv' in x))
data_df['buy'] = data_df[3].map(lambda x: int('buy' in x))
data_df['cart'] = data_df[3].map(lambda x: int('cart' in x))
data_df['favourite'] = data_df[3].map(lambda x: int('fav' in x))

In [ ]:
data_df.drop(3, axis=1, inplace=True)

In [ ]:
def filte(df: pd.DataFrame, field_name, target_field='purchase', thresh=5):
    items = set()
    df = df.groupby(by=field_name, as_index=False)[target_field].agg("sum")
    for k, v in df.values:
        if v < thresh:
            items.add(k)

    return items

In [ ]:
ban_users = filte(data_df, 0, target_field='buy', thresh=10)
ban_items = filte(data_df, 1, target_field='buy', thresh=10)

In [ ]:
drop_user = data_df[0].isin(ban_users)
data_df.drop(data_df.index[drop_user], inplace=True)

In [ ]:
drop_item = data_df[1].isin(ban_items)
data_df.drop(data_df.index[drop_item], inplace=True)

In [ ]:
data_df.rename(columns={
    0: 'user_id:token',
    1: 'item_id:token',
    2: 'category:token',
    'click': 'click:label',
    'buy': 'buy:label',
    'cart': 'cart:label',
    'favourite': 'favourite:label'
}, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
lbe = LabelEncoder()

In [ ]:
data_df["user_id:token"] = lbe.fit_transform(data_df["user_id:token"]) + 1
data_df["item_id:token"] = lbe.fit_transform(data_df["item_id:token"]) + 1
data_df["category:token"] = lbe.fit_transform(data_df["category:token"]) + 1

In [ ]:
data_df

In [ ]:
df_train, df_val = train_test_split(data_df, test_size=0.2)

In [ ]:
df_val, df_test = train_test_split(df_val, test_size=0.5)

In [ ]:
df_train.to_csv("UserBehavior_train.csv", index=False)
df_val.to_csv("UserBehavior_val.csv", index=False)
df_test.to_csv("UserBehavior_test.csv", index=False)